In [1]:
import os
import glob
import pandas as pd
import json
from omegaconf import OmegaConf

In [7]:
few_shot_folders = glob.glob('../../../paper_output/output_few_shot_dev/*_dataset_dev_*_*_shot_*')
zero_shot_folders = glob.glob('../../../paper_output/output_zero_shot_dev/*_dataset_dev_*_zero_shot_*')

# coyne_folders = glob.glob('../../../paper_output/output_few_shot_dev/*_dataset_dev_*_2_shot_coyne*')

In [9]:
for f in few_shot_folders:
    if 'command' in f:
        print(os.path.basename(f))

fce_dataset_dev_command_1_shot_0_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_1_shot_1_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_1_shot_2_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_2_shot_0_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_2_shot_1_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_2_shot_2_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_3_shot_0_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_3_shot_1_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_3_shot_2_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_4_shot_0_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_4_shot_1_temp=0.1_topk=50_topp=1.0
fce_dataset_dev_command_4_shot_2_temp=0.1_topk=50_topp=1.0
jfleg_dataset_dev_command_1_shot_0_temp=0.1_topk=50_topp=1.0
jfleg_dataset_dev_command_1_shot_1_temp=0.1_topk=50_topp=1.0
jfleg_dataset_dev_command_1_shot_2_temp=0.1_topk=50_topp=1.0
jfleg_dataset_dev_command_2_shot_0_temp=0.1_topk=50_topp=1.0
jfleg_dataset_dev_command_2_shot_1_temp=0.1_topk

In [10]:
len(few_shot_folders), len(zero_shot_folders)

(351, 212)

In [11]:
def convert_error_row_to_dict(error_info):
    return {
        'Category': error_info[0],
        'TP': error_info[1],
        'FP': error_info[2],
        'FN': error_info[3],
        'P': error_info[4],
        'R': error_info[5],
        'F0.5': error_info[6],
    }

In [12]:
columns = ['model', 'prompt_type', 'prompt_index', 'split', 'Category', 'TP', 'FP', 'FN', 'P', 'R', 'F0.5']

def extract_error_information(folders: list):
    rows = []
    for folder in folders:
        sub_dir = os.path.join(folder, 'run_1')
        folder_name = os.path.basename(folder)
        parts = folder_name.split('_')

        if not os.path.exists(os.path.join(folder, "config_detailed.yaml")):
            continue

        config = OmegaConf.load(os.path.join(folder, "config_detailed.yaml"))

        if 'coyne' in parts:
            prompt_type = '2-shot-Coyne'
            prompt_index = 1
        else:
            prompt_type = config.prompt_type
            prompt_type = '0-shot' if 'zero' in prompt_type else prompt_type
            prompt_index = int(config.prompt_index) + 1
        
        model = config.model

        split = f'{parts[0]}-{parts[2]}'

        file = os.path.join(sub_dir, 'all_results_hyp_post_errantv2-3-3.json')
        if not os.path.exists(file):
            continue
        with open(file, 'r') as f:
            data = json.load(f)['per_error_results']
        
        for d in data[1:]:
            d = convert_error_row_to_dict(d)
            rows.append([model, prompt_type, prompt_index, split, d['Category'], d['TP'], d['FP'], d['FN'], d['P'], d['R'], d['F0.5']])

    return rows    

In [13]:
# create dataframe
df = pd.DataFrame(extract_error_information(few_shot_folders), columns=columns)
df.head()

,model,prompt_type,prompt_index,split,Category,TP,FP,FN,P,R,F0.5
0,bigscience/bloomz-7b1,1-shot,1,fce-dev,M:ADJ,0,0,10,1.0,0.0,0.0
1,bigscience/bloomz-7b1,1-shot,1,fce-dev,M:ADV,1,4,26,0.2,0.037,0.1064
2,bigscience/bloomz-7b1,1-shot,1,fce-dev,M:CONJ,0,0,16,1.0,0.0,0.0
3,bigscience/bloomz-7b1,1-shot,1,fce-dev,M:CONTR,0,0,1,1.0,0.0,0.0
4,bigscience/bloomz-7b1,1-shot,1,fce-dev,M:DET,16,14,198,0.5333,0.0748,0.2395


In [14]:
df.model.unique()

array(['bigscience/bloomz-7b1', 'command', 'tiiuae/falcon-40b-instruct',
       'google/flan-t5-xxl', 'gpt-3.5-turbo-0613',
       'Writer/InstructPalmyra-20b', 'meta-llama/Llama-2-70b-chat-hf',
       'facebook/opt-iml-max-30b', 'stabilityai/StableBeluga2'],
      dtype=object)

In [15]:
dfzero = pd.DataFrame(extract_error_information(zero_shot_folders), columns=columns)

dfzero.head()

,model,prompt_type,prompt_index,split,Category,TP,FP,FN,P,R,F0.5
0,bigscience/bloomz-7b1,0-shot,11,jfleg-dev,M:ADJ,0,1,3,0.0,0.0,0.0
1,bigscience/bloomz-7b1,0-shot,11,jfleg-dev,M:ADV,1,0,8,1.0,0.1111,0.3846
2,bigscience/bloomz-7b1,0-shot,11,jfleg-dev,M:CONJ,1,0,12,1.0,0.0769,0.2941
3,bigscience/bloomz-7b1,0-shot,11,jfleg-dev,M:CONTR,1,0,1,1.0,0.5,0.8333
4,bigscience/bloomz-7b1,0-shot,11,jfleg-dev,M:DET,30,3,52,0.9091,0.3659,0.7009


In [16]:
# filter df to a specific model/prompt
dfzero[(dfzero['model']=='command') & (dfzero['prompt_type']=='0-shot') & (dfzero['split']=='jfleg-dev') & (dfzero['prompt_index']==10)]

,model,prompt_type,prompt_index,split,Category,TP,FP,FN,P,R,F0.5
1824,command,0-shot,10,jfleg-dev,M:ADJ,1,5,4,0.1667,0.2,0.1724
1825,command,0-shot,10,jfleg-dev,M:ADV,3,8,11,0.2727,0.2143,0.2586
1826,command,0-shot,10,jfleg-dev,M:CONJ,3,5,14,0.375,0.1765,0.3061
1827,command,0-shot,10,jfleg-dev,M:CONTR,2,1,0,0.6667,1.0,0.7143
1828,command,0-shot,10,jfleg-dev,M:DET,71,29,35,0.71,0.6698,0.7016
1829,command,0-shot,10,jfleg-dev,M:NOUN,5,7,6,0.4167,0.4545,0.4237
1830,command,0-shot,10,jfleg-dev,M:NOUN:POSS,4,0,0,1.0,1.0,1.0
1831,command,0-shot,10,jfleg-dev,M:OTHER,9,46,43,0.1636,0.1731,0.1654
1832,command,0-shot,10,jfleg-dev,M:PART,0,2,0,0.0,1.0,0.0
1833,command,0-shot,10,jfleg-dev,M:PREP,18,12,25,0.6,0.4186,0.5521


In [17]:
dfzero.prompt_index.unique()

array([11,  3,  5,  6,  7,  8, 10])

In [12]:
# dfcoyne = pd.DataFrame(extract_error_information(coyne_folders), columns=columns)
# # set prompt_type to 'few-shot'
# dfcoyne.prompt_type = '2-shot-Coyne'

# # select only rows where prompt_index in [1, 6] for Coyne et al replication
# dfcoyne = dfcoyne[(dfcoyne['prompt_index'].isin([1,6]))]

# # drop rows where prompt_index == 1 and model is in model_subset
# # because these models use different prompt structure for Coyne et al
# model_subset = ['gpt-3.5-turbo-0613', 'stabilityai/StableBeluga2', 'Writer/InstructPalmyra-20b', 'meta-llama/Llama-2-70b-chat-hf']
# dfcoyne = dfcoyne[~((dfcoyne['prompt_index'] == 1) & (dfcoyne['model'].isin(model_subset)))]

# # drop rows where prompt_index == 6 and model is not in model_subset
# # dfcoyne = dfcoyne[~((dfcoyne['prompt_index'] == 6) & (~dfcoyne['model'].isin(model_subset)))]

# # create new column for prompt_type and prompt_index
# # dfcoyne['prompt_type_index'] = dfcoyne['model'] + '_' + dfcoyne['prompt_type'] + '_' + dfcoyne['prompt_index'].astype(str)

# dfcoyne.head()

,model,prompt_type,prompt_index,split,Category,TP,FP,FN,P,R,F0.5
0,tiiuae/falcon-40b-instruct,2-shot-Coyne,1,wibea-dev,M:ADJ,5,29,14,0.1471,0.2632,0.1613
1,tiiuae/falcon-40b-instruct,2-shot-Coyne,1,wibea-dev,M:ADV,8,82,24,0.0889,0.25,0.102
2,tiiuae/falcon-40b-instruct,2-shot-Coyne,1,wibea-dev,M:CONJ,7,82,16,0.0787,0.3043,0.0923
3,tiiuae/falcon-40b-instruct,2-shot-Coyne,1,wibea-dev,M:CONTR,0,3,2,0.0,0.0,0.0
4,tiiuae/falcon-40b-instruct,2-shot-Coyne,1,wibea-dev,M:DET,206,170,158,0.5479,0.5659,0.5514


In [18]:
# concatenate all dataframes
df = pd.concat([df, dfzero])

In [19]:
df.prompt_type.unique()

array(['1-shot', '2-shot', '3-shot', '4-shot', '2-shot-Coyne', '0-shot'],
      dtype=object)

In [20]:
df.model.value_counts()

model
gpt-3.5-turbo-0613                3240
tiiuae/falcon-40b-instruct        3233
command                           3232
meta-llama/Llama-2-70b-chat-hf    3231
stabilityai/StableBeluga2         3229
Writer/InstructPalmyra-20b        3212
bigscience/bloomz-7b1             3203
google/flan-t5-xxl                3197
facebook/opt-iml-max-30b          3192
gpt-4-0613                        1128
Name: count, dtype: int64

In [21]:
dfcopy = df.copy()
# select columns: model, prompt_type, prompt_index, split
dfcopy = dfcopy[['model', 'prompt_type', 'prompt_index', 'split']]
# drop duplicates
dfcopy = dfcopy.drop_duplicates()

dfcopy.model.value_counts()

model
bigscience/bloomz-7b1             60
command                           60
tiiuae/falcon-40b-instruct        60
google/flan-t5-xxl                60
gpt-3.5-turbo-0613                60
Writer/InstructPalmyra-20b        60
meta-llama/Llama-2-70b-chat-hf    60
facebook/opt-iml-max-30b          60
stabilityai/StableBeluga2         60
gpt-4-0613                        21
Name: count, dtype: int64

In [22]:
for m in df.model.unique():
    if 'Llama' not in m:
        continue
    print(m)
    
    # filter to a split
    dft = df[df.model == m]
    dft = dft[dft.split == 'jfleg-dev']

    # get value_counts for each split and prompt_type
    print(dft[dft.model == m].groupby(['split', 'prompt_type', 'prompt_index']).count()['model'])

meta-llama/Llama-2-70b-chat-hf
split      prompt_type   prompt_index
jfleg-dev  0-shot        3               54
                         5               54
                         6               53
                         7               53
                         8               54
                         10              53
                         11              52
           1-shot        1               53
                         2               53
                         3               54
           2-shot        1               53
                         2               53
                         3               54
           2-shot-Coyne  1               54
           3-shot        1               54
                         2               54
                         3               54
           4-shot        1               54
                         2               54
                         3               54
Name: model, dtype: int64


In [24]:
df[df.model == 'meta-llama/Llama-2-70b-chat-hf'].groupby(['prompt_type', 'prompt_index']).count()

model  split  Category   TP   FP   FN    P    R  \
prompt_type  prompt_index                                                    
0-shot       3               162    162       162  162  162  162  162  162   
             5               162    162       162  162  162  162  162  162   
             6               161    161       161  161  161  161  161  161   
             7               161    161       161  161  161  161  161  161   
             8               162    162       162  162  162  162  162  162   
             10              161    161       161  161  161  161  161  161   
             11              160    160       160  160  160  160  160  160   
1-shot       1               161    161       161  161  161  161  161  161   
             2               161    161       161  161  161  161  161  161   
             3               162    162       162  162  162  162  162  162   
2-shot       1               161    161       161  161  161  161  161  161   
             2               161    161       161  161  161  161  161  161   
             3               162    162       162  162  162  162  162  162   
2-shot-Coyne 1               162    162       162  162  162  162  162  162   
3-shot       1               162    162       162  162  162  162  162  162   
             2               162    162       162  162  162  162  162  162   
             3               162    162       162  162  162  162  162  162   
4-shot       1               162    162       162  162  162  162  162  162   
             2               162    162       162  162  162  162  162  162   
             3               162    162       162  162  162  162  162  162   

                           F0.5  
prompt_type  prompt_index        
0-shot       3              162  
             5              162  
             6              161  
             7              161  
             8              162  
             10             161  
             11             160  
1-shot       1              161  
             2              161  
             3              162  
2-shot       1              161  
             2              161  
             3              162  
2-shot-Coyne 1              162  
3-shot       1              162  
             2              162  
             3              162  
4-shot       1              162  
             2              162  
             3              162

In [25]:
# save dataframe
df.to_csv('../all_error_type_results.csv', index=False)